# Токенизација

> Токенизација је процес представљања сирових података у виду токена (најчеће бајтови података)

Од квалитета токена доста зависи и учинак самог модела
Токен је најмањи вид податка коме модел даје значење, и пошто их може бити ограничен број, јер цена модела расте значајно са повећањем самог вокабулара, избор величине токена и шта тај токен треба да обухвати је доста тежак задатак, и мале одлуке доста могу утицати на то шта ће модел разумети из података. 


### Токенизација текста

Код токенизације текста, најчешће се текст у виду стрига прво претвори у бајтове помоћу utf-8

In [2]:
text = "Здраво" 
text_bytes = text.encode("utf-8")

print(text, text_bytes)
print(len(text), len(text_bytes))


Здраво b'\xd0\x97\xd0\xb4\xd1\x80\xd0\xb0\xd0\xb2\xd0\xbe'
6 12


- Међутим овде настаје проблем, пошто је utf-8 погодан само за латинична слова, __ћирилица__ се претвори у 2 бајта и због алгоритма који користимо за спајање токена, број токена које будемо генерисали ће бити дупло већи и сами токени ће бити мање ефикасни

- Због овог разлога ћемо користити сам текст

In [8]:
text = "Здраво"
text_ints = [ord(s) for s in text]

print (text, text_ints)
print (len(text), len(text_ints))

Здраво [1047, 1076, 1088, 1072, 1074, 1086]
6 6


### Byte-Pair Encoding

Овај алгоритам и две методе које се понављају

1. Пронађи два суседна слова која се понављају најчешће

2. Замини да два пара са новим токеном

Ове две методе се понављају над текстом за тренинг токенизера док се не добије жељени број токена

In [ ]:
vocabulary = {} # Вокабулар (int) -> ('char')

INPUT_FILE = "D:\Caslav\Poso\AI\EpskiGPT\data\\narodne_pesme.txt"

# Функција за учитавањње почетног скупа знакова
def create_vocabulary(text: str):
    chars = sorted(list(set(text)))
    vocab = { i:ch for i,ch in enumerate(chars) }
    return vocab

# Отварање фајла из ког вадимо текст
with open(INPUT_FILE, "r") as file:
    text = file.read()

vocabulary = create_vocabulary(text)

print(vocabulary)

{0: '\n', 1: ' ', 2: '!', 3: "'", 4: '(', 5: ')', 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'â', 14: 'ê', 15: 'ô', 16: '̓', 17: 'Ђ', 18: 'Ј', 19: 'Љ', 20: 'Њ', 21: 'Ћ', 22: 'Џ', 23: 'А', 24: 'Б', 25: 'В', 26: 'Г', 27: 'Д', 28: 'Е', 29: 'Ж', 30: 'З', 31: 'И', 32: 'К', 33: 'Л', 34: 'М', 35: 'Н', 36: 'О', 37: 'П', 38: 'Р', 39: 'С', 40: 'Т', 41: 'У', 42: 'Ф', 43: 'Х', 44: 'Ц', 45: 'Ч', 46: 'Ш', 47: 'а', 48: 'б', 49: 'в', 50: 'г', 51: 'д', 52: 'е', 53: 'ж', 54: 'з', 55: 'и', 56: 'к', 57: 'л', 58: 'м', 59: 'н', 60: 'о', 61: 'п', 62: 'р', 63: 'с', 64: 'т', 65: 'у', 66: 'ф', 67: 'х', 68: 'ц', 69: 'ч', 70: 'ш', 71: 'ђ', 72: 'ј', 73: 'љ', 74: 'њ', 75: 'ћ', 76: 'џ', 77: '–', 78: '—', 79: '’', 80: '“', 81: '”', 82: '„'}


In [40]:
# Поделићемо тренирање Токенајзера у неколико корака

# Помоћна функција која броји понаљање узастопних токена
def count_conseq_tokens(ids, counts={}):
    """
    За дату листу интиџера, врати речник броја понављања узаступних парова
    Пример: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    Опционо ажурира већ дата пребројавања
    """
    for pair in zip(ids[:-1], ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

count_conseq_tokens([1, 2, 3, 1, 2], {(1, 2): 2})


{(1, 2): 4, (2, 3): 1, (3, 1): 1}

In [45]:
# Функција која обацује нове токене у текст
def merge(ids, pair, idx):
    """
    Замени све парове pair који се појављују у ids листи са idx
    Пример: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
    """
    new_ids = []
    i = 0
    ids_n = len(ids)
    while i < ids_n-1:
        if (ids[i], ids[i+1]) == pair:
            new_ids.append(idx)
            i+=1
        else:
            new_ids.append(ids[i])
        i+=1
    
    return new_ids

merge([1, 2, 3, 1, 2], (1, 2), 4)


[4, 3, 4]